In [3]:
# 📈 LSTM Stock Forecast (STL + LSTM)
import streamlit as st
import yfinance as yf
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.keras.layers import LSTM, Dense
from tensorflow.keras.models import Sequential
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from statsmodels.tsa.seasonal import STL
from datetime import datetime

# 🎨 Streamlit Config
st.set_page_config(page_title="Stock Forecast (LSTM + STL)", layout="wide")

st.title("📊 LSTM Forecast on Stock High Prices (STL Decomposition)")
st.markdown("### Hybrid Model: STL (Trend + Seasonal) + LSTM")

# 📅 Input Section
ticker = st.text_input("Enter Stock Symbol (e.g. HDFCBANK.NS):", "HDFCBANK.NS")
start_date = st.date_input("Start Date:", datetime(2020, 1, 1))
end_date_forecast = st.date_input("End Date:", datetime(2025, 8, 6))

if st.button("🚀 Run Forecast"):
    # 1️⃣ Download Data
    st.write("Fetching data from Yahoo Finance...")
    data = yf.download(ticker, start=start_date, end=end_date_forecast)
    d_high = data["High"]

    st.write(f"**Total Records:** {len(data)}")
    st.line_chart(d_high, use_container_width=True)

    # 2️⃣ STL Decomposition
    st.write("### STL Decomposition (Trend + Seasonal + Residual)")
    stl = STL(d_high, period=30)
    result = stl.fit()
    trend = result.trend
    seasonal = result.seasonal
    resid = result.resid

    fig, axs = plt.subplots(3, 1, figsize=(10, 6), sharex=True)
    axs[0].plot(trend, label='Trend', color='orange')
    axs[0].legend()
    axs[1].plot(seasonal, label='Seasonal', color='green')
    axs[1].legend()
    axs[2].plot(resid, label='Residual', color='red')
    axs[2].legend()
    st.pyplot(fig)

    # 3️⃣ Prepare LSTM Data
    def prepare_lstm_data(series, window_size):
        scaler = MinMaxScaler()
        scaled = scaler.fit_transform(series.values.reshape(-1, 1))
        X, y = [], []
        for i in range(len(scaled) - window_size):
            X.append(scaled[i:i + window_size])
            y.append(scaled[i + window_size])
        return np.array(X), np.array(y), scaler

    window_size = 10
    X_trend, y_trend, scaler_trend = prepare_lstm_data(trend, window_size)
    X_seasonal, y_seasonal, scaler_seasonal = prepare_lstm_data(seasonal, window_size)

    # 4️⃣ LSTM Builder
    def build_and_train_lstm(X, y, epochs=10):
        model = Sequential([
            LSTM(5, return_sequences=True, input_shape=(X.shape[1], 1)),
            LSTM(5, return_sequences=True),
            LSTM(5, return_sequences=True),
            LSTM(5, return_sequences=False),
            Dense(1)
        ])
        model.compile(optimizer='adam', loss='mse')
        model.fit(X, y, epochs=epochs, batch_size=10, verbose=0)
        return model

    st.write("### Training LSTM models (Trend + Seasonal)...")
    model_trend = build_and_train_lstm(X_trend, y_trend)
    model_seasonal = build_and_train_lstm(X_seasonal, y_seasonal)

    # 5️⃣ Predictions
    y_trend_pred = model_trend.predict(X_trend)
    y_seasonal_pred = model_seasonal.predict(X_seasonal)

    trend_pred = scaler_trend.inverse_transform(y_trend_pred)
    seasonal_pred = scaler_seasonal.inverse_transform(y_seasonal_pred)

    final_pred = trend_pred.flatten() + seasonal_pred.flatten()
    actual = d_high.values[window_size:]

    # 6️⃣ Plot Actual vs Predicted
    st.write("### Actual vs Predicted (Trend + Seasonal Reconstruction)")
    fig2, ax2 = plt.subplots(figsize=(10, 5))
    ax2.plot(actual, label="Actual")
    ax2.plot(final_pred, label="Predicted (Trend + Seasonal)")
    ax2.set_title("LSTM Forecast on STL Components")
    ax2.legend()
    ax2.grid(True)
    st.pyplot(fig2)

    # 7️⃣ RMSE
    rmse = np.sqrt(mean_squared_error(actual, final_pred))
    st.metric("📉 RMSE (Reconstructed Forecast)", f"{rmse:.4f}")

    # 8️⃣ Forecast Next Day
    last_trend_scaled = scaler_trend.transform(trend.values[-window_size:].reshape(-1, 1)).reshape(1, window_size, 1)
    next_trend_scaled = model_trend.predict(last_trend_scaled)
    next_trend = scaler_trend.inverse_transform(next_trend_scaled)[0][0]

    last_seasonal_scaled = scaler_seasonal.transform(seasonal.values[-window_size:].reshape(-1, 1)).reshape(1, window_size, 1)
    next_seasonal_scaled = model_seasonal.predict(last_seasonal_scaled)
    next_seasonal = scaler_seasonal.inverse_transform(next_seasonal_scaled)[0][0]

    next_day_forecast = next_trend + next_seasonal
    st.success(f"📅 **Forecasted High Price for Next Day:** {next_day_forecast:.2f}")


2025-10-16 04:41:49.066 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 04:41:49.068 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 04:41:49.659 
  command:

    streamlit run /usr/local/lib/python3.12/dist-packages/colab_kernel_launcher.py [ARGUMENTS]
2025-10-16 04:41:49.661 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 04:41:49.662 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 04:41:49.663 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-10-16 04:41:49.664 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when runn

In [2]:
pip install streamlit

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 10.1/10.1 MB 97.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.9/6.9 MB 114.3 MB/s eta 0:00:00
